## 문제 1. 
- 다음은 Travel Insurance 데이터 세트이다. 주어진 훈련 데이터 세트를 이용하여 고객별 여행보험 가입 여부 예측 모형을 만들고, 가장 높은 Accuracy 값을 가지는 최종 모델을 도출하시오. 해당 모델을 활용하여 보험 가입 여부 예측값을 계산하고 결괏값은 CSV 파일로 제출하시오.
- 결과 제출 양식 : 제출한 예측값의 ROC_AUC 점수 결과를 통해 영역별 배점에 따라 최종 점수가 반영될 예정

In [1]:
import pandas as pd 
pd.DataFrame({
    "ID"  : [0, 1, 2], 
    "y_pred" : [0.2543, 0.1324, 0.5892]
})

,ID,y_pred
0,0,0.2543
1,1,0.1324
2,2,0.5892


## 데이터 불러오기

In [2]:
import pandas as pd 
X_test = pd.read_csv("data/3회/304_x_test.csv")
X_train = pd.read_csv("data/3회/304_x_train.csv")
y_train = pd.read_csv("data/3회/304_y_train.csv")

In [3]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   ID               1490 non-null   int64
 1   TravelInsurance  1490 non-null   int64
dtypes: int64(2)
memory usage: 23.4 KB


## 데이터 정보

In [4]:
import pandas as pd 
pd.DataFrame({
    "변수" : list(X_train.columns) + ["TravelInsurance"], 
    "설명" : [
        "고객 ID", 
        "고객 나이", 
        "고객 직업 유형", 
        "대졸 여부", 
        "연 소득", 
        "가족 수",
        "만성질환 여부", 
        "FrequentFlyer 자격 여부", 
        "해외여행 경험",   
        "여행보험 가입 여부(0: 미가입, 1: 가입)"
    ]
})

,변수,설명
0,ID,고객 ID
1,Age,고객 나이
2,Employment Type,고객 직업 유형
3,GraduateOrNot,대졸 여부
4,AnnualIncome,연 소득
5,FamilyMembers,가족 수
6,ChronicDiseases,만성질환 여부
7,FrequentFlyer,FrequentFlyer 자격 여부
8,EverTravelledAbroad,해외여행 경험
9,TravelInsurance,"여행보험 가입 여부(0: 미가입, 1: 가입)"


## 데이터 확인

In [5]:
X_train.head(1)

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,1704,26,Private Sector/Self Employed,Yes,1400000,3,1,No,Yes


In [6]:
X_test.head(1)

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,1569,27,Government Sector,Yes,500000,5,0,No,No


In [7]:
y_train.head(1)

,ID,TravelInsurance
0,1704,0


## 데이터 확인

### 각 컬럼의 정보 확인

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
dtypes: int64(5), object(4)
memory usage: 104.9+ KB


In [9]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   497 non-null    int64 
 1   Age                  497 non-null    int64 
 2   Employment Type      497 non-null    object
 3   GraduateOrNot        497 non-null    object
 4   AnnualIncome         497 non-null    int64 
 5   FamilyMembers        497 non-null    int64 
 6   ChronicDiseases      497 non-null    int64 
 7   FrequentFlyer        497 non-null    object
 8   EverTravelledAbroad  497 non-null    object
dtypes: int64(5), object(4)
memory usage: 35.1+ KB


### 결측치 확인
- 결측치는 존재하지 않는 것 확인

In [10]:
X_train.isnull().sum()

ID                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [11]:
X_test.isnull().sum()

ID                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

### 컬럼 삭제

In [12]:
X_train_id = X_train.pop('ID')
X_test_id = X_test.pop('ID')

X_train.shape, X_test.shape

((1490, 8), (497, 8))

## 종속변수 데이터셋 확인

In [13]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   ID               1490 non-null   int64
 1   TravelInsurance  1490 non-null   int64
dtypes: int64(2)
memory usage: 23.4 KB


In [14]:
y_train['TravelInsurance'].value_counts()

TravelInsurance
0    950
1    540
Name: count, dtype: int64

- 비율 차이가 존재하기 때문에, stratify 옵션을 넣어준다. 

### 데이터 확인

In [15]:
X_train.describe()

,Age,AnnualIncome,FamilyMembers,ChronicDiseases
count,1490.000000,1.490000e+03,1490.000000,1490.000000
mean,29.668456,9.386577e+05,4.732215,0.285235
std,2.935416,3.731487e+05,1.612608,0.451678
min,25.000000,3.000000e+05,2.000000,0.000000
25%,28.000000,6.000000e+05,4.000000,0.000000
50%,29.000000,9.000000e+05,5.000000,0.000000
75%,32.000000,1.250000e+06,6.000000,1.000000
max,35.000000,1.800000e+06,9.000000,1.000000


## 데이터 처리
- ChronicDiseases는 OneHotEncoder로 처리해야 한다. 
- `AnnualIncome`은 RobustScaler로 처리한다.
- 그 외 나머지는 StandardScaler로 처리한다.
- 문자열은 모두 one-hot encoder로 처리한다. 

In [16]:
X_train['ChronicDiseases'] = X_train['ChronicDiseases'].astype(str)
X_test['ChronicDiseases'] = X_test['ChronicDiseases'].astype(str)

- 문자 데이터만 추출

In [17]:
import numpy as np
object_df = X_train.select_dtypes(exclude=np.number)
object_df.head(3)

,Employment Type,GraduateOrNot,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,Private Sector/Self Employed,Yes,1,No,Yes
1,Private Sector/Self Employed,Yes,1,No,No
2,Private Sector/Self Employed,Yes,0,No,Yes


- 수치 데이터만 추출

In [18]:
numbers_df = X_train.select_dtypes(include=np.number)
numbers_df.columns

Index(['Age', 'AnnualIncome', 'FamilyMembers'], dtype='object')

In [19]:
object_df['Employment Type'].value_counts()

Employment Type
Private Sector/Self Employed    1070
Government Sector                420
Name: count, dtype: int64

## 데이터셋 분리

In [20]:
from sklearn.model_selection import train_test_split 
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train['TravelInsurance'], 
    stratify=y_train['TravelInsurance'],
    random_state=42, 
    test_size=0.25
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((1117, 8), (373, 8), (1117,), (373,))

## 모형 학습 및 예측값

In [22]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

from lightgbm import LGBMClassifier
#from sklearn.utils.fixes import loguniform
from scipy.stats import loguniform

param_grid = {
    'clf__learning_rate': loguniform(0.001, 0.1),
    'clf__n_estimators': np.arange(50, 200),
    'clf__max_depth':  np.arange(3, 30, 3),
    'clf__num_leaves': np.arange(20, 50),
    "clf__min_split_gain" : np.arange(0, 1.1, 0.1), ## called gamma
    'clf__subsample': np.arange(0.6, 1.0, 0.1)
}

column_transformer = ColumnTransformer([
    ("robust_scaler", MinMaxScaler(), ['AnnualIncome']),
    ("std_scaler", StandardScaler(), ['Age', 'FamilyMembers']),
    ("ohc_encoder", OneHotEncoder(), object_df.columns)
], remainder="passthrough")

pipeline = Pipeline([
    ("preprocessor", column_transformer),
    ("clf", LGBMClassifier(random_state=42))
])

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=50,  
    scoring='roc_auc',  
    cv=5,  
    verbose=500,
    n_jobs=-1  
)

# 모형학습
random_search.fit(X_tr, y_tr)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Info] Number of positive: 405, number of negative: 712
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000253 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 70
[LightGBM] [Info] Number of data points in the train set: 1117, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.362578 -> initscore=-0.564191
[LightGBM] [Info] Start training from score -0.564191
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('robust_scaler',
                                                                               MinMaxScaler(),
                                                                               ['AnnualIncome']),
                                                                              ('std_scaler',
                                                                               StandardScaler(),
                                                                               ['Age',
                                                                                'FamilyMembers']),
                                                                              ('ohc_encoder',
                                                                               OneHotEncoder(),
                                                                               Index(['Employment Type', 'GraduateOrNot', 'ChronicDiseases', 'FrequentFlyer',
       'EverTravelledAbr...
       154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
       180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192,
       193, 194, 195, 196, 197, 198, 199]),
                                        'clf__num_leaves': array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                                        'clf__subsample': array([0.6, 0.7, 0.8, 0.9])},
                   scoring='roc_auc', verbose=500)

## 평가지표 구현
- 평가지표를 구하는 코드를 작성한다. 

In [23]:
def get_scores(model, X_tr, X_val, y_tr, y_val):
    y_tr_pred = model.predict_proba(X_tr)[:, 1]
    y_val_pred = model.predict_proba(X_val)[:, 1]
    tr_score = roc_auc_score(y_tr, y_tr_pred)
    val_score = roc_auc_score(y_val, y_val_pred)
    return f'train : {round(tr_score, 3)}, valid : {round(val_score, 4)}'
get_scores(random_search, X_tr, X_val, y_tr, y_val)

'train : 0.817, valid : 0.811'

In [24]:
best_params = random_search.best_params_
best_score = random_search.best_score_
print('Best Parameters:', best_params)
print('Best Score:', best_score)

Best Parameters: {'clf__learning_rate': 0.005323926261993663, 'clf__max_depth': 3, 'clf__min_split_gain': 0.4, 'clf__n_estimators': 109, 'clf__num_leaves': 27, 'clf__subsample': 0.7999999999999999}
Best Score: 0.8104542475434494


## 결과 제출

In [25]:
final_preds = random_search.predict_proba(X_test)
result = pd.DataFrame({"ID" : X_test_id, "pred": final_preds[:, 1]})
result.head(1)

,ID,pred
0,1569,0.293108


In [26]:
result.to_csv("00000.csv", index=False)